In [1]:
%matplotlib inline
import os
import gzip
import random
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

/srv/soft/python/versions/python-2.7.6/lib/python2.7/site-packages/pandas/io/excel.py:626: UserWarning: Installed openpyxl is not supported at this time. Use >=1.6.1 and <2.0.0.
  .format(openpyxl_compat.start_ver, openpyxl_compat.stop_ver))


In [2]:
TIDS_GREATER_THAN_10 = "./uniq_tid_full_chembl20_cutoff800_sorted_by_mid_tid_greatequal10count.csv.gz"
FULL_DATASET = "../full_chembl20_cutoff800_sorted_by_mid_tid.csv.gz"
OUT_FILE = "../full_chembl20_cutoff800_sorted_by_mid_tid_greaterthan10tids.csv.gz"
RAND_CHOICE = "../full_chembl20_cutoff800_sorted_by_mid_tid_greaterthan10tids_rand_choice.csv.gz"

RANDOM_STATE = 42

In [3]:
chembl_tids = set()
with gzip.open(TIDS_GREATER_THAN_10, "r") as f:
    for line in f: 
        chembl_tids.update({line.split("\t")[1]})

In [4]:
# write all data where >10 targets
chembl_mids = set()
with gzip.open(FULL_DATASET, "r") as f:
    with gzip.open(OUT_FILE, "w") as out:
        # print header line
        header = f.readline()
        out.write(header)
        print header
        for line in f:
            if line.split("\t")[2] in chembl_tids:
                out.write(line)
                chembl_mids.update({line.split("\t")[3]})

doc_id	year	ChEMBL_Target_ID	ChEMBL_Molecule_ID	Chembl_Target_pref_name	Activity (nM)	Relation	SMILES



In [5]:
tid_lookup = {t:i for i,t in enumerate(list(chembl_tids))}
mid_lookup = {t:i for i,t in enumerate(list(chembl_mids))}

In [6]:
adj_arry = np.zeros((len(chembl_tids), len(chembl_mids)))
with gzip.open(OUT_FILE, "r") as f:
    print f.readline()
    for line in f:
        curr_tid = line.split("\t")[2]
        curr_mid = line.split("\t")[3]
        if curr_tid in chembl_tids:
            adj_arry[tid_lookup[curr_tid]][mid_lookup[curr_mid]] += 1

doc_id	year	ChEMBL_Target_ID	ChEMBL_Molecule_ID	Chembl_Target_pref_name	Activity (nM)	Relation	SMILES



In [7]:
vals = adj_arry[adj_arry>0]

In [8]:
np.where(adj_arry == vals.max())

(array([1878]), array([249249]))

In [9]:
for key, idx in tid_lookup.iteritems():
    if idx == 1878:
        print key
for key, idx in mid_lookup.iteritems():
    if idx == 249249:
        print key

CHEMBL205
CHEMBL20


### general  note: the most overrepresented is (carbonic anhydrase,Acetazolamide) with 249 instances

In [10]:
vals.max()

249.0

In [11]:
# get a dict of all training examples and their indices
ex_locs = {}
loc = 0
with gzip.open(OUT_FILE, "r") as f: 
    # print header line
    header = f.readline()
    print header
    loc += 1
    for line in f: 
        curr_tid = line.split("\t")[2]
        curr_mid = line.split("\t")[3]
        if curr_tid in chembl_tids:
            if (curr_tid, curr_mid) in ex_locs:
                ex_locs[(curr_tid, curr_mid)].append(loc)
            else:
                ex_locs[(curr_tid, curr_mid)] = [loc]
            loc += 1

doc_id	year	ChEMBL_Target_ID	ChEMBL_Molecule_ID	Chembl_Target_pref_name	Activity (nM)	Relation	SMILES



In [13]:
# Now, choose one training set randomly
random.seed(RANDOM_STATE)
to_write = set()
all_vals = []
for val_lists in ex_locs.itervalues():
    all_vals.append(len(val_lists))
    to_write.update({random.choice(val_lists)})

last_mid = ""
last_tid = ""
loc = 0
with gzip.open(OUT_FILE, "r") as f: 
    with gzip.open(RAND_CHOICE, "w") as out:
        # print header line
        header = f.readline()
        print header
        out.write(header)
        loc += 1
        for line in f:
            if loc in to_write:
                out.write(line)

doc_id	year	ChEMBL_Target_ID	ChEMBL_Molecule_ID	Chembl_Target_pref_name	Activity (nM)	Relation	SMILES

